# Search the perfect home

## Mandatory points


* family: most important important thing that is mandatory
  * max time = 20 minutes 
* work: 
    * train stations 
        * 30 minutes by bike
        * by foot:  replaced by bike
        * union trains
    * work place
      * max 20 minutes by car

## Ponderation definition

* area of interests
* aeras of nuisance



In [37]:
import folium
from openrouteservice import client
from geopy.geocoders import Nominatim
import geopandas as gpd
import yaml

In [38]:
locations = {
    'family1': {
        'address': "rue de floreffe, spy",
        'type': 'family',
        'time': 60 * 20.0,
        'profile': 'driving-car',
        'weight': 1.0
    },
    'office1': {
        'address': "Rue des Linottes, 5100 Namur",
        'type': 'office',
        'time': 60 * 20.0,
        'profile': 'driving-car',
        'weight': 1.0
    },
    'train_station1': {
        'address': "Gare de Namur",
        'type': 'train',
        'profile': 'cycling-regular',
        'time': 60 * 30.0,
        'weight': .5
    },
    'train_station2': {
        'address': 'Gare de Gembloux',
        'type': 'train',
        'time': 60 * 30.0,
        'profile': 'cycling-regular',
        'weight': .5
    }
}


In [39]:
# Search places
geolocator = Nominatim(user_agent="folium")
for loc, attr in locations.items():
    geocode = geolocator.geocode(attr['address'])
    locations[loc]['address'] = geocode
    locations[loc]['latitude'] = geocode.latitude
    locations[loc]['longitude'] = geocode.longitude
df = pd.DataFrame.from_dict(locations, orient='index')
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs=4326
)
gdf

,address,type,time,profile,weight,latitude,longitude,geometry
family1,"(Rue de Floreffe, Spy, Jemeppe-sur-Sambre, Nam...",family,1200.0,driving-car,1.0,50.475269,4.718138,POINT (4.71814 50.47527)
office1,"(Rue des Linottes, Naninne, Namur, Wallonie, 5...",office,1200.0,driving-car,1.0,50.420205,4.932417,POINT (4.93242 50.42020)
train_station1,"(Cup Pasta, Place de la Station, Bomel, Namur,...",train,1800.0,cycling-regular,0.5,50.468648,4.862911,POINT (4.86291 50.46865)
train_station2,"(Gare, Chaussée de Nivelles, Mazy, Gembloux, N...",train,1800.0,cycling-regular,0.5,50.512928,4.675656,POINT (4.67566 50.51293)


In [40]:
# create isochrnone

with open(r'tokens.yaml') as file:
    tokens = yaml.load(file, Loader=yaml.FullLoader)
ors_token = tokens['ors']
clnt = client.Client(key=ors_token) 
locs = []
for loc, attr in gdf.iterrows():
    params_iso = {
        'profile': attr['profile'],
        'range': [attr['time']],
        }
    params_iso['locations'] = [[ attr['geometry'].x, attr['geometry'].y]] # Add location coords to request parameters
    locs.append(clnt.isochrones(**params_iso)) # Perform isochrone request
print(locs)

[{'type': 'FeatureCollection', 'bbox': [4.355211, 50.350066, 5.111124, 50.631024], 'features': [{'type': 'Feature', 'properties': {'group_index': 0, 'value': 1200.0, 'center': [4.7181380036147536, 50.47526889015872]}, 'geometry': {'coordinates': [[[4.355211, 50.479743], [4.357322, 50.479467], [4.370076, 50.477734], [4.38058, 50.476303], [4.388537, 50.475216], [4.396495, 50.474128], [4.400352, 50.473615], [4.404204, 50.473103], [4.414533, 50.467071], [4.414266, 50.463942], [4.414302, 50.46358], [4.416481, 50.460636], [4.417049, 50.460043], [4.417578, 50.45956], [4.420006, 50.462218], [4.417857, 50.465687], [4.424297, 50.474463], [4.432639, 50.472559], [4.434446, 50.470138], [4.439552, 50.463146], [4.439928, 50.460034], [4.439956, 50.460022], [4.441328, 50.460015], [4.441521, 50.460125], [4.442299, 50.464872], [4.444162, 50.467068], [4.44778, 50.469171], [4.448266, 50.469443], [4.454316, 50.459089], [4.45427, 50.45549], [4.454348, 50.455489], [4.461236, 50.459264], [4.47102, 50.456673], 

In [30]:
# prepare map colors
clr = ('green', 'gray', 'white', 'orange', 'red', 'darkgreen', 'lightgray', 'darkpurple', 'pink', 'cadetblue', 'purple', 'lightgreen', 'blue', 'beige', 'black', 'lightblue', 'darkblue', 'darkred', 'lightred')

for i, x in enumerate(gdf.index):
    color = clr[i]
    gdf.at[x, 'color'] = color

# convert GeoJSON collection list into one collection
# convert GeoJson into GeoPandas Data Frame
from shapely.geometry import shape
geom = []
for loc in locs:
    geom.append(shape(loc['features'][0]['geometry']))
gdf2 = gdf.set_geometry(geom, drop=True, crs=4326)

gdf2 = gdf2.drop(columns=['address'])
gdf2


,type,time,profile,weight,latitude,longitude,geometry,color
family1,family,1200.0,driving-car,1.0,50.478005,4.718279,"POLYGON ((4.33922 50.48337, 4.33953 50.47978, ...",green
office1,office,1200.0,driving-car,1.0,50.420785,4.932067,"POLYGON ((4.65064 50.49611, 4.65064 50.49251, ...",gray
train_station1,train,1800.0,cycling-regular,0.5,50.468648,4.862911,"POLYGON ((4.74361 50.48825, 4.74393 50.48465, ...",white
train_station2,train,1800.0,cycling-regular,0.5,50.512928,4.675656,"POLYGON ((4.55650 50.53294, 4.55729 50.53251, ...",orange


In [32]:
# Set up folium map

def style(feature):
    #print(feature)
    return {
        'fillColor': feature['properties']['color'],
        'color': feature['properties']['color'],
        'weight': 1
    }

map1 = folium.Map(tiles='OpenStreetMap', location=(gdf['geometry'].get('family1').y, gdf['geometry'].get('family1').x), zoom_start=11)

for loc, ser in gdf.iterrows():
    folium.map.Marker(location=[ser['geometry'].y, ser['geometry'].x], # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(
                          color=ser['color'],
                          icon_color='#cc0000',
                          icon='home',
                          prefix='fa',
                      ),
                      popup=loc,
                      ).add_to(map1) # Add apartment locations to map
#for loc, ser in gdf2.iterrows():
folium.GeoJson(
    gdf2.to_json(),
    name='GeoJson',
    style_function=style
).add_to(map1) # Add GeoJson to map

map1

# To-Do

* Ajouter une double zone tampon autour de spy
* Ajouter spy au dessus des autres
* Fusionner les zones par type